In [225]:
import os
import uuid
import pandas as pd
from PIL import Image
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.transforms import functional as TF

from timm.data.mixup import Mixup
import albumentations as A
from augraphy import AugraphyPipeline, VoronoiTessellation

# --------------------------------------------------------------------------
# 1. 기본 설정
# --------------------------------------------------------------------------
# 원본 데이터 경로
ORIGINAL_CSV_PATH = './data/train.csv'
ORIGINAL_IMG_DIR = './data/train/'

# 최종적으로 생성될 데이터 폴더 경로
FINAL_TRAIN_DIR = './data/train_augmented_v2/'
FINAL_VAL_DIR = './data/val_augmented_v2/'

# 증강 및 분리 관련 파라미터
VAL_SIZE = 0.5
RANDOM_STATE = 42
BATCH_SIZE = 32
NUM_CLASSES = 17
AUGMENTATION_COUNT = 100 # 데이터셋 당 적용할 증강 횟수

# --------------------------------------------------------------------------
# 2. 데이터셋 클래스 및 초기 변환 (재사용)
# --------------------------------------------------------------------------
class ResizeWithPadding:
    """
    이미지 비율을 유지하며 리사이즈하고, 남는 공간을 패딩으로 채웁니다.
    """
    def __init__(self, size, fill=(0, 0, 0)):
        self.size = size
        self.fill = fill

    def __call__(self, image):
        w, h = image.size
        scale = min(self.size[0] / w, self.size[1] / h)
        new_w, new_h = int(w * scale), int(h * scale)
        resized = TF.resize(image, (new_h, new_w))
        pad_left = (self.size[0] - new_w) // 2
        pad_top = (self.size[1] - new_h) // 2
        pad_right = self.size[0] - new_w - pad_left
        pad_bottom = self.size[1] - new_h - pad_top
        return TF.pad(resized, (pad_left, pad_top, pad_right, pad_bottom), fill=self.fill)

initial_transform = T.Compose([
ResizeWithPadding((591, 443), fill=(255, 255, 255)),
T.ToTensor(),
])

class CustomImageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['ID']
        label = self.df.iloc[idx]['target']
        image_path = os.path.join(self.image_dir, img_name)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# --------------------------------------------------------------------------
# 3. 핵심 기능 함수 (증강 및 저장 로직 통합)
# --------------------------------------------------------------------------

def augment_and_save_dataset(df, base_dir, csv_name, description):
    """
    주어진 데이터프레임에 증강을 적용하고 지정된 경로에 이미지와 CSV를 저장합니다.
    """
    print(f"\n🚀 {description} 시작...")

    # --- 경로 설정 및 폴더 생성 ---
    output_img_dir = os.path.join(base_dir, 'images')
    output_csv_path = os.path.join(base_dir, csv_name)
    os.makedirs(output_img_dir, exist_ok=True)

    # --- 증강 파이프라인 정의 ---
    mixup_fn = Mixup(mixup_alpha=0.5, cutmix_alpha=0.0, prob=1.0, label_smoothing=0.1, num_classes=NUM_CLASSES)
    augraphy_pipeline = AugraphyPipeline([
    VoronoiTessellation(num_cells_range=(150, 250), p=0.5)
    ])
    albumentations_transform = A.Compose([
    # A.GaussianBlur(blur_limit=(5, 7), p=1.0),
    A.Rotate(limit=360, p=0.9, border_mode=0, value=(255, 255, 255)),
    ])

    # --- 데이터 로딩 및 증강 ---
    dataset = CustomImageDataset(df=df, image_dir=ORIGINAL_IMG_DIR, transform=initial_transform)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE,drop_last=True)

    new_metadata = []
    for images, labels in tqdm(dataloader, desc=description):
        for _ in range(AUGMENTATION_COUNT):
            if images.size(0) % 2 != 0:
                images = torch.cat([images, images[-1].unsqueeze(0)], dim=0)
                labels = torch.cat([labels, labels[-1].unsqueeze(0)], dim=0)
            mixed_images, mixed_labels = mixup_fn(images, labels)
            for i in range(mixed_images.size(0)):
                img_tensor, label_array = mixed_images[i], mixed_labels[i].cpu().numpy()
                img_np = np.array(TF.to_pil_image(img_tensor))

                # Augraphy -> Albumentations 적용
                augmented_np = albumentations_transform(image=augraphy_pipeline(image=img_np))["image"]

                final_image_pil = Image.fromarray(augmented_np)
                filename = f"{uuid.uuid4()}.png"
                final_image_pil.save(os.path.join(output_img_dir, filename))

                new_metadata.append({'ID': filename, 'target': str(list(label_array))})

    df_new = pd.DataFrame(new_metadata)
    df_new.to_csv(output_csv_path, index=False)
    print(f"✅ 완료: {len(df_new)}개의 증강 이미지가 '{output_img_dir}'에 저장되었습니다.")

# --------------------------------------------------------------------------
# 4. 메인 실행 블록
# --------------------------------------------------------------------------

# --- 1. 데이터 분리 ---
print("📊 원본 데이터를 학습용과 검증용으로 분리합니다.")
df_original = pd.read_csv(ORIGINAL_CSV_PATH)

df_train, df_val = train_test_split(
df_original,
test_size=VAL_SIZE,
shuffle=True,
random_state=RANDOM_STATE,
stratify=df_original['target']
)
print(f"분리 결과 -> 학습용: {len(df_train)}개, 검증용: {len(df_val)}개")

# --- 2. 학습 데이터 증강 및 저장 ---
augment_and_save_dataset(
    df=df_train,
    base_dir=FINAL_TRAIN_DIR,
    csv_name='train_augmented_v2.csv',
    description="학습 데이터 증강"
)

# --- 3. 검증 데이터 증강 및 저장 ---
augment_and_save_dataset(
    df=df_val,
    base_dir=FINAL_VAL_DIR,
    csv_name='val_augmented_v2.csv',
    description="검증 데이터 증강"
)

print("\n🎉 모든 작업이 완료되었습니다!")

📊 원본 데이터를 학습용과 검증용으로 분리합니다.
분리 결과 -> 학습용: 785개, 검증용: 785개

🚀 학습 데이터 증강 시작...


학습 데이터 증강:   0%|          | 0/24 [00:00<?, ?it/s]

학습 데이터 증강: 100%|██████████| 24/24 [3:24:02<00:00, 510.11s/it]  


✅ 완료: 76800개의 증강 이미지가 './data/train_augmented_v2/images'에 저장되었습니다.

🚀 검증 데이터 증강 시작...


검증 데이터 증강: 100%|██████████| 24/24 [3:24:09<00:00, 510.40s/it]  


✅ 완료: 76800개의 증강 이미지가 './data/val_augmented_v2/images'에 저장되었습니다.

🎉 모든 작업이 완료되었습니다!


In [204]:
import os
import uuid
import pandas as pd
from PIL import Image
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.transforms import functional as TF

from timm.data.mixup import Mixup
import albumentations as A
from augraphy import AugraphyPipeline, VoronoiTessellation

# --------------------------------------------------------------------------
# 1. 기본 설정
# --------------------------------------------------------------------------
# 원본 데이터 경로
ORIGINAL_CSV_PATH = './data/train.csv'
ORIGINAL_IMG_DIR = './data/train/'

# 최종적으로 생성될 데이터 폴더 경로
FINAL_TRAIN_DIR = './data/train_augmented_v2/'
FINAL_VAL_DIR = './data/val_augmented_v2/'

# 증강 및 분리 관련 파라미터
VAL_SIZE = 0.5
RANDOM_STATE = 42
BATCH_SIZE = 32
NUM_CLASSES = 17
AUGMENTATION_COUNT = 100 # 데이터셋 당 적용할 증강 횟수

# --------------------------------------------------------------------------
# 2. 데이터셋 클래스 및 초기 변환 (재사용)
# --------------------------------------------------------------------------
class ResizeWithPadding:
    """
    이미지 비율을 유지하며 리사이즈하고, 남는 공간을 패딩으로 채웁니다.
    """
    def __init__(self, size, fill=(0, 0, 0)):
        self.size = size
        self.fill = fill

    def __call__(self, image):
        w, h = image.size
        scale = min(self.size[0] / w, self.size[1] / h)
        new_w, new_h = int(w * scale), int(h * scale)
        resized = TF.resize(image, (new_h, new_w))
        pad_left = (self.size[0] - new_w) // 2
        pad_top = (self.size[1] - new_h) // 2
        pad_right = self.size[0] - new_w - pad_left
        pad_bottom = self.size[1] - new_h - pad_top
        return TF.pad(resized, (pad_left, pad_top, pad_right, pad_bottom), fill=self.fill)

initial_transform = T.Compose([
    ResizeWithPadding((591, 443), fill=(255, 255, 255)),
    T.ToTensor(),
])

class CustomImageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['ID']
        label = self.df.iloc[idx]['target']
        image_path = os.path.join(self.image_dir, img_name)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# --------------------------------------------------------------------------
# 3. 핵심 기능 함수 (증강 및 저장 로직 통합)
# --------------------------------------------------------------------------

def augment_and_save_dataset(df, base_dir, csv_name, description):
    """
    주어진 데이터프레임에 증강을 적용하고 지정된 경로에 이미지와 CSV를 저장합니다.
    """
    print(f"\n🚀 {description} 시작...")

    # --- 경로 설정 및 폴더 생성 ---
    output_img_dir = os.path.join(base_dir, 'images')
    output_csv_path = os.path.join(base_dir, csv_name)
    os.makedirs(output_img_dir, exist_ok=True)

    # --- 증강 파이프라인 정의 ---
    mixup_fn = Mixup(mixup_alpha=0.5, cutmix_alpha=0.0, prob=1.0, label_smoothing=0.1, num_classes=NUM_CLASSES)
    augraphy_pipeline = AugraphyPipeline([
        VoronoiTessellation(num_cells_range=(150, 250), p=0.5)
    ])
    albumentations_transform = A.Compose([
        # A.GaussianBlur(blur_limit=(5, 7), p=1.0),
        A.Rotate(limit=360, p=0.9, border_mode=0, value=(255, 255, 255)),
    ])

    # --- 데이터 로딩 및 증강 ---
    dataset = CustomImageDataset(df=df, image_dir=ORIGINAL_IMG_DIR, transform=initial_transform)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZEArithmeticError)

    new_metadata = []
    for images, labels in tqdm(dataloader, desc=description):
        for _ in range(AUGMENTATION_COUNT):
            if images.size(0) % 2 != 0:
                # 마지막 이미지와 라벨을 복제하여 이어 붙임
                images = torch.cat([images, images[-1].unsqueeze(0)], dim=0)
                labels = torch.cat([labels, labels[-1].unsqueeze(0)], dim=0)
            mixed_images, mixed_labels = mixup_fn(images, labels)
            for i in range(mixed_images.size(0)):
                img_tensor, label_array = mixed_images[i], mixed_labels[i].cpu().numpy()
                img_np = np.array(TF.to_pil_image(img_tensor))

                # Augraphy -> Albumentations 적용
                augmented_np = albumentations_transform(image=augraphy_pipeline(image=img_np))["image"]

                final_image_pil = Image.fromarray(augmented_np)
                filename = f"{uuid.uuid4()}.png"
                final_image_pil.save(os.path.join(output_img_dir, filename))

                new_metadata.append({'ID': filename, 'target': str(list(label_array))})

    df_new = pd.DataFrame(new_metadata)
    df_new.to_csv(output_csv_path, index=False)
    print(f"✅ 완료: {len(df_new)}개의 증강 이미지가 '{output_img_dir}'에 저장되었습니다.")

# --------------------------------------------------------------------------
# 4. 메인 실행 블록
# --------------------------------------------------------------------------
if __name__ == '__main__':
    # --- 1. 데이터 분리 ---
    print("📊 원본 데이터를 학습용과 검증용으로 분리합니다.")
    df_original = pd.read_csv(ORIGINAL_CSV_PATH)

    df_train, df_val = train_test_split(
        df_original,
        test_size=VAL_SIZE,
        shuffle=True,
        random_state=RANDOM_STATE,
        stratify=df_original['target']
    )
    print(f"분리 결과 -> 학습용: {len(df_train)}개, 검증용: {len(df_val)}개")

    # --- 2. 학습 데이터 증강 및 저장 ---
    augment_and_save_dataset(
        df=df_train,
        base_dir=FINAL_TRAIN_DIR,
        csv_name='train_augmented_v2.csv',
        description="학습 데이터 증강"
    )

    # --- 3. 검증 데이터 증강 및 저장 ---
    augment_and_save_dataset(
        df=df_val,
        base_dir=FINAL_VAL_DIR,
        csv_name='val_augmented_v2.csv',
        description="검증 데이터 증강"
    )

    print("\n🎉 모든 작업이 완료되었습니다!")

📊 원본 데이터를 학습용과 검증용으로 분리합니다.
분리 결과 -> 학습용: 785개, 검증용: 785개

🚀 학습 데이터 증강 시작...


학습 데이터 증강:   0%|          | 0/24 [00:00<?, ?it/s]

학습 데이터 증강: 100%|██████████| 24/24 [02:15<00:00,  5.64s/it]


✅ 완료: 768개의 증강 이미지가 './data/train_augmented_v2/images'에 저장되었습니다.

🚀 검증 데이터 증강 시작...


검증 데이터 증강: 100%|██████████| 24/24 [02:12<00:00,  5.54s/it]

✅ 완료: 768개의 증강 이미지가 './data/val_augmented_v2/images'에 저장되었습니다.

🎉 모든 작업이 완료되었습니다!


In [223]:
a= pd.read_csv('./data/train_augmented_v2/train_augmented_v2.csv')

In [224]:
import ast
import pandas as pd
a = pd.read_csv('data/train_augmented_v2/train_augmented_v2.csv')
a['target'] = a['target'].apply(lambda x : ast.literal_eval(x))
a['c'] = a['target'].apply(lambda x : x.index(max(x)))
a['c'].value_counts()

c
16    50
10    50
8     50
11    50
7     49
15    49
2     49
3     49
9     49
4     49
12    48
6     47
5     47
0     47
13    37
14    25
1     23
Name: count, dtype: int64